In [ ]:
# ============================================================================
# COMPREHENSIVE PLOTBOT DIAGNOSTIC
# Run this entire cell and share ALL output
# ============================================================================

import sys
import os
from datetime import datetime

print("="*80)
print("COMPREHENSIVE PLOTBOT DIAGNOSTIC")
print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

# ============================================================================
# SECTION 1: ENVIRONMENT & IMPORTS
# ============================================================================
print("\n" + "="*80)
print("1. ENVIRONMENT & IMPORTS")
print("="*80)

try:
    print(f"Python version: {sys.version}")
    print(f"Python executable: {sys.executable}")
    print(f"Current working directory: {os.getcwd()}")
    print(f"SPEDAS_DATA_DIR env var: {os.environ.get('SPEDAS_DATA_DIR', 'NOT SET')}")
except Exception as e:
    print(f"❌ Error getting environment info: {e}")

# Import core packages
print("\nImporting core packages...")
packages_status = {}

for pkg_name in ['numpy', 'pandas', 'matplotlib', 'cdflib', 'pyspedas']:
    try:
        pkg = __import__(pkg_name)
        version = getattr(pkg, '__version__', 'unknown')
        packages_status[pkg_name] = f"✅ {version}"
    except ImportError as e:
        packages_status[pkg_name] = f"❌ Not installed: {e}"
    except Exception as e:
        packages_status[pkg_name] = f"⚠️  Import error: {e}"

for pkg, status in packages_status.items():
    print(f"  {pkg:15s}: {status}")

# Import plotbot
print("\nImporting plotbot...")
try:
    import plotbot
    from plotbot import mag_rtn_4sa, epad
    import numpy as np
    import matplotlib.pyplot as plt
    print("  ✅ Plotbot imported successfully")
except Exception as e:
    print(f"  ❌ Failed to import plotbot: {e}")
    import traceback
    traceback.print_exc()
    print("\n⛔ Cannot continue without plotbot - stopping here")
    raise SystemExit("Plotbot import failed")

# ============================================================================
# SECTION 2: PLOTBOT CONFIGURATION
# ============================================================================
print("\n" + "="*80)
print("2. PLOTBOT CONFIGURATION")
print("="*80)

try:
    from plotbot import config
    print(f"config.data_dir:        {config.data_dir}")
    print(f"config.data_server:     {config.data_server}")
    print(f"Data dir exists:        {os.path.exists(config.data_dir)}")
    
    # Check if data directory is writable
    if os.path.exists(config.data_dir):
        test_file = os.path.join(config.data_dir, '.plotbot_write_test')
        try:
            with open(test_file, 'w') as f:
                f.write('test')
            os.remove(test_file)
            print(f"Data dir writable:      ✅ Yes")
        except Exception as e:
            print(f"Data dir writable:      ❌ No - {e}")
    else:
        print(f"Data dir writable:      ⚠️  Directory doesn't exist")
        # Try to create it
        try:
            os.makedirs(config.data_dir, exist_ok=True)
            print(f"  → Created data directory: {config.data_dir}")
        except Exception as e:
            print(f"  → ❌ Cannot create data directory: {e}")
except Exception as e:
    print(f"❌ Error reading config: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 3: DATA PATHS & FILES
# ============================================================================
print("\n" + "="*80)
print("3. DATA PATHS & EXISTING FILES")
print("="*80)

try:
    from plotbot.data_classes.data_types import data_types, get_local_path
    import glob
    
    # Check MAG data
    print("\nMAG (mag_rtn_4sa) data:")
    mag_config = data_types.get('mag_rtn_4sa')
    if mag_config:
        mag_path = get_local_path('mag_rtn_4sa')
        print(f"  Expected path: {mag_path}")
        mag_files = glob.glob(os.path.join(mag_path, '**', '*.cdf'), recursive=True)
        print(f"  Found {len(mag_files)} CDF files")
        if mag_files:
            print(f"  Sample: {os.path.basename(mag_files[0])}")
    
    # Check EPAD data
    print("\nEPAD (epad) data:")
    epad_config = data_types.get('spe_sf0_pad')
    if epad_config:
        epad_path = get_local_path('spe_sf0_pad').format(data_level=epad_config['data_level'])
        print(f"  Expected path: {epad_path}")
        epad_files = glob.glob(os.path.join(epad_path, '**', '*.cdf'), recursive=True)
        print(f"  Found {len(epad_files)} CDF files")
        if epad_files:
            print(f"  Sample: {os.path.basename(epad_files[0])}")
    
    # Search for 2020-01-29 files specifically
    print("\nSearching for 2020-01-29 data files:")
    jan29_mag = glob.glob(os.path.join(config.data_dir, '**', '*20200129*.cdf'), recursive=True)
    jan29_mag = [f for f in jan29_mag if 'mag' in f.lower()]
    print(f"  MAG files for 2020-01-29: {len(jan29_mag)}")
    
    jan29_epad = glob.glob(os.path.join(config.data_dir, '**', '*20200129*.cdf'), recursive=True)
    jan29_epad = [f for f in jan29_epad if 'spe' in f.lower()]
    print(f"  EPAD files for 2020-01-29: {len(jan29_epad)}")
    
except Exception as e:
    print(f"❌ Error checking data paths: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 4: PLOTBOT CALL TEST
# ============================================================================
print("\n" + "="*80)
print("4. PLOTBOT CALL TEST")
print("="*80)

trange = ['2020-01-29/18:00:00', '2020-01-29/20:00:00']
print(f"\nTest time range: {trange}")
print(f"Calling plotbot with mag_rtn_4sa.br, .bn, and epad.strahl...\n")

# Enable some debug output
try:
    plotbot.print_manager.show_status = False
    plotbot.print_manager.show_debug = False
    plotbot.print_manager.show_data_cubby = False
except:
    pass

plotbot_call_success = False
try:
    fig = plotbot.plotbot(trange, mag_rtn_4sa.br, 1, mag_rtn_4sa.bn, 2, epad.strahl, 3)
    print("✅ Plotbot call completed without error")
    plotbot_call_success = True
    plt.close(fig)  # Close the figure to avoid display
except Exception as e:
    print(f"❌ Plotbot call failed: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 5: DATA ACCESS & DATETIME CHECK (THE KEY TEST!)
# ============================================================================
print("\n" + "="*80)
print("5. DATA ACCESS & DATETIME VALIDATION")
print("="*80)
print("\nThis section tests the EXACT pattern you sent to your collaborator...\n")

try:
    # Test accessing mag_rtn_4sa.br
    print("Testing mag_rtn_4sa.br:")
    print("-" * 40)
    
    # Check if data property returns anything
    br_data = None
    br_datetime = None
    
    try:
        br_data = np.array(mag_rtn_4sa.br.data)
        print(f"  br.data shape:          {br_data.shape}")
        print(f"  br.data length:         {len(br_data)}")
        print(f"  br.data has content:    {len(br_data) > 0}")
        if len(br_data) > 0:
            print(f"  br.data[0]:             {br_data[0]}")
            print(f"  br.data dtype:          {br_data.dtype}")
    except Exception as e:
        print(f"  ❌ Error accessing br.data: {e}")
    
    try:
        br_datetime = np.array(mag_rtn_4sa.br.datetime_array)
        print(f"\n  datetime_array shape:   {br_datetime.shape}")
        print(f"  datetime_array length:  {len(br_datetime)}")
        print(f"  datetime has content:   {len(br_datetime) > 0}")
        
        if len(br_datetime) > 0:
            print(f"  datetime_array[0]:      {br_datetime[0]}")
            print(f"  datetime_array[-1]:     {br_datetime[-1]}")
            print(f"  datetime type:          {type(br_datetime[0])}")
            print(f"  datetime dtype:         {br_datetime.dtype}")
            
            # Check if we have the 1970 issue
            first_dt = br_datetime[0]
            if hasattr(first_dt, 'year'):
                if first_dt.year == 1970:
                    print(f"\n  ⚠️  WARNING: Detected 1970 date! This is THE ISSUE!")
                    print(f"      First datetime: {first_dt}")
                    print(f"      Expected year: 2020")
                    print(f"      Actual year: {first_dt.year}")
                else:
                    print(f"\n  ✅ Datetime looks correct (year={first_dt.year})")
        else:
            print(f"\n  ❌ CRITICAL: datetime_array is EMPTY!")
            print(f"      This is why you're seeing 1970 dates!")
            print(f"      matplotlib is plotting empty/zero values as epoch time")
    except Exception as e:
        print(f"  ❌ Error accessing datetime_array: {e}")
        import traceback
        traceback.print_exc()
    
    # Test mag_rtn_4sa.bn
    print("\nTesting mag_rtn_4sa.bn:")
    print("-" * 40)
    try:
        bn_data = np.array(mag_rtn_4sa.bn.data)
        print(f"  bn.data length:         {len(bn_data)}")
        bn_datetime = np.array(mag_rtn_4sa.bn.datetime_array)
        print(f"  datetime_array length:  {len(bn_datetime)}")
        if len(bn_datetime) > 0:
            print(f"  datetime_array[0]:      {bn_datetime[0]}")
    except Exception as e:
        print(f"  ❌ Error: {e}")
    
    # Test the actual calculation pattern
    print("\nTesting your exact calculation pattern:")
    print("-" * 40)
    try:
        tB = np.array(mag_rtn_4sa.br.datetime_array)
        Br = np.array(mag_rtn_4sa.br.data)
        Bn = np.array(mag_rtn_4sa.bn.data)
        
        print(f"  tB length:              {len(tB)}")
        print(f"  Br length:              {len(Br)}")
        print(f"  Bn length:              {len(Bn)}")
        
        if len(tB) > 0 and len(Br) > 0 and len(Bn) > 0:
            phi_B = np.degrees(np.arctan2(Br, Bn)) + 180
            print(f"  phi_B calculated:       ✅ Success")
            print(f"  phi_B length:           {len(phi_B)}")
            print(f"  phi_B range:            [{phi_B.min():.1f}, {phi_B.max():.1f}] degrees")
            
            # Try creating a test plot
            print(f"\n  Creating test scatter plot...")
            fig, ax = plt.subplots(1, 1, figsize=(10, 4))
            ax.scatter(tB, phi_B, s=3)
            ax.set_ylabel(r'$\phi_B \ (\circ)$')
            
            # Get x-axis limits to see what dates are being plotted
            xlim = ax.get_xlim()
            print(f"  Plot x-axis limits:     {xlim}")
            
            # Convert to dates if possible
            try:
                from matplotlib.dates import num2date
                date_min = num2date(xlim[0])
                date_max = num2date(xlim[1])
                print(f"  Plot date range:        {date_min} to {date_max}")
                
                if date_min.year == 1970:
                    print(f"\n  ❌ CONFIRMED: Plot is showing 1970 dates!")
                    print(f"      This means matplotlib is receiving bad datetime values")
                else:
                    print(f"\n  ✅ Plot dates look correct (year={date_min.year})")
            except:
                pass
            
            plt.close(fig)
        else:
            print(f"  ❌ Cannot calculate phi_B - one or more arrays are empty!")
            if len(tB) == 0:
                print(f"      → tB (datetime_array) is empty")
            if len(Br) == 0:
                print(f"      → Br (data) is empty")
            if len(Bn) == 0:
                print(f"      → Bn (data) is empty")
    except Exception as e:
        print(f"  ❌ Error during calculation: {e}")
        import traceback
        traceback.print_exc()
        
except Exception as e:
    print(f"❌ Error in data access section: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 6: DATA CUBBY INSPECTION
# ============================================================================
print("\n" + "="*80)
print("6. DATA CUBBY INSPECTION")
print("="*80)

try:
    from plotbot.data_cubby import data_cubby
    
    print(f"\nData cubby keys: {list(data_cubby.cubby.keys())}")
    
    # Check mag_rtn_4sa in cubby
    if 'mag_rtn_4sa' in data_cubby.cubby:
        mag_instance = data_cubby.cubby['mag_rtn_4sa']
        print(f"\nmag_rtn_4sa in cubby:")
        if hasattr(mag_instance, 'datetime_array'):
            dt_len = len(mag_instance.datetime_array) if mag_instance.datetime_array is not None else 0
            print(f"  datetime_array length: {dt_len}")
            if dt_len > 0:
                print(f"  datetime_array[0]:     {mag_instance.datetime_array[0]}")
        if hasattr(mag_instance, 'raw_data'):
            if mag_instance.raw_data:
                print(f"  raw_data keys:         {list(mag_instance.raw_data.keys())}")
                if 'br' in mag_instance.raw_data:
                    br_len = len(mag_instance.raw_data['br']) if mag_instance.raw_data['br'] is not None else 0
                    print(f"  raw_data['br'] length: {br_len}")
    
    # Check epad in cubby
    for key in ['epad', 'spe_sf0_pad']:
        if key in data_cubby.cubby:
            epad_instance = data_cubby.cubby[key]
            print(f"\n{key} in cubby:")
            if hasattr(epad_instance, 'datetime_array'):
                dt_len = len(epad_instance.datetime_array) if epad_instance.datetime_array is not None else 0
                print(f"  datetime_array length: {dt_len}")
                if dt_len > 0:
                    print(f"  datetime_array[0]:     {epad_instance.datetime_array[0]}")
            break
            
except Exception as e:
    print(f"❌ Error inspecting data cubby: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 7: INTERNAL STATE CHECK
# ============================================================================
print("\n" + "="*80)
print("7. INTERNAL STATE CHECK")
print("="*80)

try:
    print("\nmag_rtn_4sa.br internal state:")
    print(f"  Type:                   {type(mag_rtn_4sa.br)}")
    print(f"  Has _clipped_data:      {hasattr(mag_rtn_4sa.br, '_clipped_data')}")
    print(f"  Has _clipped_datetime:  {hasattr(mag_rtn_4sa.br, '_clipped_datetime_array')}")
    
    if hasattr(mag_rtn_4sa.br, '_clipped_data'):
        clipped = mag_rtn_4sa.br._clipped_data
        if clipped is not None:
            print(f"  _clipped_data length:   {len(clipped)}")
        else:
            print(f"  _clipped_data:          None")
    
    if hasattr(mag_rtn_4sa.br, '_clipped_datetime_array'):
        clipped_dt = mag_rtn_4sa.br._clipped_datetime_array
        if clipped_dt is not None:
            print(f"  _clipped_datetime len:  {len(clipped_dt)}")
            if len(clipped_dt) > 0:
                print(f"  _clipped_datetime[0]:   {clipped_dt[0]}")
        else:
            print(f"  _clipped_datetime:      None")
    
    if hasattr(mag_rtn_4sa.br, 'plot_config'):
        print(f"  Has plot_config:        ✅ Yes")
        if hasattr(mag_rtn_4sa.br.plot_config, 'datetime_array'):
            pc_dt = mag_rtn_4sa.br.plot_config.datetime_array
            if pc_dt is not None:
                print(f"  plot_config.dt length:  {len(pc_dt)}")
                if len(pc_dt) > 0:
                    print(f"  plot_config.dt[0]:      {pc_dt[0]}")
            else:
                print(f"  plot_config.dt:         None")
    
    if hasattr(mag_rtn_4sa.br, 'requested_trange'):
        print(f"  requested_trange:       {mag_rtn_4sa.br.requested_trange}")
        
except Exception as e:
    print(f"❌ Error checking internal state: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 8: DIAGNOSIS SUMMARY
# ============================================================================
print("\n" + "="*80)
print("8. DIAGNOSIS SUMMARY")
print("="*80)

print("\n📋 KEY FINDINGS:")
print("-" * 40)

# Check each potential issue
issues_found = []
things_working = []

try:
    # Issue 1: Data directory
    if not os.path.exists(config.data_dir):
        issues_found.append("Data directory doesn't exist")
    else:
        things_working.append("Data directory exists")
    
    # Issue 2: Plotbot call
    if not plotbot_call_success:
        issues_found.append("Plotbot call failed with error")
    else:
        things_working.append("Plotbot call succeeded")
    
    # Issue 3: Empty data
    if 'br_data' in locals() and br_data is not None:
        if len(br_data) == 0:
            issues_found.append("br.data is EMPTY - no data loaded!")
        else:
            things_working.append(f"br.data has {len(br_data)} points")
    
    # Issue 4: Empty datetime
    if 'br_datetime' in locals() and br_datetime is not None:
        if len(br_datetime) == 0:
            issues_found.append("datetime_array is EMPTY - this causes 1970 dates!")
        else:
            things_working.append(f"datetime_array has {len(br_datetime)} points")
            
            # Issue 5: 1970 dates
            if hasattr(br_datetime[0], 'year'):
                if br_datetime[0].year == 1970:
                    issues_found.append(f"Datetime shows 1970 instead of 2020! ({br_datetime[0]})")
                else:
                    things_working.append(f"Datetime year is correct ({br_datetime[0].year})")
except Exception as e:
    issues_found.append(f"Error during summary: {e}")

if things_working:
    print("\n✅ WORKING:")
    for item in things_working:
        print(f"   • {item}")

if issues_found:
    print("\n❌ ISSUES DETECTED:")
    for item in issues_found:
        print(f"   • {item}")
else:
    print("\n✅ NO ISSUES DETECTED - Everything looks good!")

print("\n" + "="*80)
print("DIAGNOSTIC COMPLETE")
print("Please copy ALL output above and share it")
print("="*80)
